# <center>Домашнее задание №2. Решение
## <center>Анализ данных о сердечно‑сосудистых заболеваниях

В этом задании вы будете отвечать на вопросы по датасету о сердечно‑сосудистых заболеваниях. Скачивать данные не нужно: они уже есть в репозитории. Некоторые задачи потребуют от вас написать код.

#### Постановка задачи

Предсказать наличие или отсутствие сердечно‑сосудистого заболевания (CVD) по результатам обследования пациента.

#### Описание данных

Есть 3 типа входных признаков:

- *Objective*: фактическая информация;
- *Examination*: результаты медицинского обследования;
- *Subjective*: информация, сообщённая пациентом.

| Feature | Variable Type | Variable      | Value Type |
|---------|--------------|---------------|------------|
| Age | Objective Feature | age | int (days) |
| Height | Objective Feature | height | int (cm) |
| Weight | Objective Feature | weight | float (kg) |
| Gender | Objective Feature | gender | categorical code |
| Systolic blood pressure | Examination Feature | ap_hi | int |
| Diastolic blood pressure | Examination Feature | ap_lo | int |
| Cholesterol | Examination Feature | cholesterol | 1: normal, 2: above normal, 3: well above normal |
| Glucose | Examination Feature | gluc | 1: normal, 2: above normal, 3: well above normal |
| Smoking | Subjective Feature | smoke | binary |
| Alcohol intake | Subjective Feature | alco | binary |
| Physical activity | Subjective Feature | active | binary |
| Presence or absence of cardiovascular disease | Target Variable | cardio | binary |

Все значения в датасете были собраны в момент медицинского обследования.

Давайте познакомимся с данными, проведя предварительный анализ.

#  Часть 1. Предварительный анализ данных

Сначала инициализируем окружение:

In [ ]:
# Import all required modules
import pandas as pd
import numpy as np

# Disable warnings
import warnings
warnings.filterwarnings("ignore")

# Import plotting modules
import seaborn as sns
sns.set()
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker

Для визуального анализа будем использовать библиотеку `seaborn`, настроим её:

In [ ]:
# Tune the visual settings for figures in `seaborn`
sns.set_context(
    "notebook", 
    font_scale=1.5,       
    rc={ 
        "figure.figsize": (11, 8), 
        "axes.titlesize": 18 
    }
)

from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8

Для простоты будем работать только с обучающей частью датасета:

In [ ]:
df = pd.read_csv('../data/mlbootcamp5_train.csv')
print('Dataset size: ', df.shape)
df.head()

Для начала полезно посмотреть на сами значения признаков.
 
Преобразуем данные в *длинный* формат и изобразим частоты значений категориальных признаков с помощью [`catplot()`](https://seaborn.pydata.org/generated/seaborn.catplot.html) (в старых версиях это делали через `factorplot`).

In [ ]:
df_uniques = pd.melt(frame=df, value_vars=['gender','cholesterol', 
                                           'gluc', 'smoke', 'alco', 
                                           'active', 'cardio'])
df_uniques = pd.DataFrame(df_uniques.groupby(['variable', 
                                              'value'])['value'].count()) \
    .sort_index(level=[0, 1]) \
    .rename(columns={'value': 'count'}) \
    .reset_index()

sns.catplot(x='variable', y='count', hue='value', 
            data=df_uniques, kind='bar', height=12);

Видно, что классы целевой переменной сбалансированы — это хорошо.

Разобьём датасет по значениям таргета: иногда уже по такому графику можно на глаз заметить самый важный признак.

In [ ]:
df_uniques = pd.melt(frame=df, value_vars=['gender','cholesterol', 
                                           'gluc', 'smoke', 'alco', 
                                           'active'], 
                     id_vars=['cardio'])
df_uniques = pd.DataFrame(df_uniques.groupby(['variable', 'value', 
                                              'cardio'])['value'].count()) \
    .sort_index(level=[0, 1]) \
    .rename(columns={'value': 'count'}) \
    .reset_index()

sns.catplot(x='variable', y='count', hue='value', 
            col='cardio', data=df_uniques, kind='bar', height=9);

Заметьте, что распределения уровня холестерина и глюкозы сильно отличаются в зависимости от значения таргета. Случайность ли это?

Теперь посчитаем некоторые статистики по уникальным значениям признаков:

In [ ]:
for c in df.columns:
    n = df[c].nunique()
    print(c)
    if n <= 3:
        print(n, sorted(df[c].value_counts().to_dict().items()))
    else:
        print(n)
    print(10 * '-')

В итоге у нас:
- 5 числовых признаков (без *id*);
- 7 категориальных признаков;
- всего 70000 записей.

## 1.1. Базовые наблюдения

**Вопрос 1.1. Сколько мужчин и женщин представлено в этом датасете? Значение признака `gender` не подписано (какое значение — женщины, какое — мужчины), определите это по росту, считая, что мужчины в среднем выше.**
1. 45530 женщин и 24470 мужчин
2. 45530 мужчин и 24470 женщин
3. 45470 женщин и 24530 мужчин
4. 45470 мужчин и 24530 женщин

**Ответ:** 1.

### Решение:

Посчитаем средний рост для обоих значений `gender`: 

In [ ]:
df.groupby('gender')['height'].mean()

161 см и почти 170 см в среднем, поэтому делаем вывод, что `gender=1` — женщины, а `gender=2` — мужчины. Итого в выборке 45530 женщин и 24470 мужчин.

**Вопрос 1.2. Какой пол чаще сообщает о потреблении алкоголя — мужчины или женщины?**
1. женщины
2. мужчины

**Ответ:** 2.

### Решение:

In [ ]:
df.groupby('gender')['alco'].mean()

Ну… очевидно :)

**Вопрос 1.3. Какова разница (в процентах, с округлением) между долей курящих среди мужчин и среди женщин?**
1. 4
2. 16
3. 20
4. 24

**Ответ:** 3.

### Решение:

In [ ]:
df.groupby('gender')['smoke'].mean()

In [ ]:
round(100 * (df.loc[df['gender'] == 2, 'smoke'].mean() - df.loc[df['gender'] == 1, 'smoke'].mean()))

**Вопрос 1.4. Какова разница между медианным возрастом курящих и некурящих (в месяцах, с округлением)? Для этого нужно понять, в каких единицах задан признак `age` в этом датасете.**

1. 5
2. 10
3. 15
4. 20

**Ответ:** 4.

### Решение:

Возраст здесь задан в днях.

In [ ]:
df.groupby('smoke')['age'].median() / 365.25

Медианный возраст курящих — 52.4 года, некурящих — 54 года. Видим, что правильный ответ — 20 месяцев. Но можно посчитать это и более точно:

In [ ]:
(df[df['smoke'] == 0]['age'].median() - 
 df[df['smoke'] == 1]['age'].median()) / 365.25 * 12

## 1.2. Карты риска
### Задача:

На сайте Европейского общества кардиологов есть [шкала SCORE](https://www.escardio.org/Education/Practice-Tools/CVD-prevention-toolbox/SCORE-Risk-Charts). Она используется для оценки риска смерти от сердечно‑сосудистых заболеваний в ближайшие 10 лет. Вот фрагмент:

<img src='../../img/SCORE_CVD_eng.png' width=60%>

Рассмотрим правый верхний прямоугольник — он соответствует курящим мужчинам в возрасте от 60 до 65 лет. (Неочевидно, но значения на картинке задают верхнюю границу интервала.)

В левом нижнем углу прямоугольника видим число 9, в правом верхнем — 47. Это означает, что для людей этой группы пола и возраста с систолическим давлением ниже 120 риск CVD примерно в 5 раз ниже, чем для тех, у кого давление в интервале [160, 180).

Посчитаем тот же коэффициент по нашим данным.

Уточнения:
- Постройте признак ``age_years`` — возраст, округлённый до целого числа лет. Для этой задачи выберите только людей 60–64 лет включительно.
- Категории уровня холестерина на картинке и в нашем датасете различаются. Перевод для признака ``cholesterol`` такой: 4 ммоль/л $\rightarrow$ 1, 5–7 ммоль/л $\rightarrow$ 2, 8 ммоль/л $\rightarrow$ 3.

**Вопрос 1.5. Посчитайте долю людей с CVD в двух описанных выше сегментах. Чему равен коэффициент (отношение) этих двух долей?**

1. 1
2. 2
3. 3
4. 4

**Ответ:** 3.

### Solution:

In [ ]:
df['age_years'] = (df['age'] / 365.25).round().astype('int')

In [ ]:
df['age_years'].max()

Самые возрастные люди в выборке — 65 лет. Совпадение? Вряд ли. Выберем курящих мужчин 60–64 лет.

In [ ]:
smoking_old_men = df[(df['gender'] == 2) & (df['age_years'] >= 60)
                    & (df['age_years'] < 65) & (df['smoke'] == 1)]

Если в этой возрастной группе уровень холестерина равен 1, а систолическое давление меньше 120, то доля людей с CVD составляет 26%.

In [ ]:
smoking_old_men[(smoking_old_men['cholesterol'] == 1) &
               (smoking_old_men['ap_hi'] < 120)]['cardio'].mean()

Если же в этой возрастной группе уровень холестерина равен 3, а систолическое давление в диапазоне [160, 180), то доля людей с CVD составляет 86%.

In [ ]:
smoking_old_men[(smoking_old_men['cholesterol'] == 3) &
               (smoking_old_men['ap_hi'] >= 160) &
               (smoking_old_men['ap_hi'] < 180)]['cardio'].mean()

В итоге отличие примерно трёхкратное. Не пятикратное, как на шкале SCORE, но вполне возможно, что риск по SCORE нелинейно зависит от доли больных в данной группе.

## 1.3. Анализ BMI
### Задача:

Создайте новый признак — BMI ([Body Mass Index](https://en.wikipedia.org/wiki/Body_mass_index), индекс массы тела). Для этого разделите вес в килограммах на квадрат роста в метрах. Нормальным обычно считается BMI в диапазоне от 18.5 до 25. 

**Вопрос 1.6. Выберите верные утверждения:**

1. Медианный BMI в выборке попадает в диапазон нормальных значений.
2. У женщин BMI в среднем выше, чем у мужчин.
3. У здоровых людей BMI в среднем выше, чем у людей с CVD.
4. У здоровых непьющих мужчин BMI ближе к норме, чем у здоровых непьющих женщин.

**Ответ:** 2 и 4.

### Решение:

In [ ]:
df['BMI'] = df['weight'] / (df['height'] / 100) ** 2

In [ ]:
df['BMI'].median()

Первое утверждение неверно: медианный BMI превышает верхнюю границу нормы 25.

In [ ]:
df.groupby('gender')['BMI'].median()

Второе утверждение верно — у женщин BMI в среднем выше.

Третье утверждение неверно.

In [ ]:
df.groupby(['gender', 'alco', 'cardio'])['BMI'].median().to_frame()

Сравнивая значения BMI в строках, где `alco = 0` и `cardio = 0`, видим, что последнее утверждение верно.

## 1.4. Очистка данных

### Задача:
Видно, что данные далеки от идеала: в них есть «грязь» и неточности. На визуализациях это проявится ещё лучше.

Отфильтруйте следующие группы пациентов (будем считать их ошибочными записями):

- диастолическое давление выше систолического; 
- рост строго меньше 2.5‑го перцентиля (используйте `pd.Series.quantile`; если функция незнакома — загляните в документацию);
- рост строго больше 97.5‑го перцентиля;
- вес строго меньше 2.5‑го перцентиля;
- вес строго больше 97.5‑го перцентиля.

Это ещё не всё, что можно сделать для очистки данных, но для наших целей этого достаточно.

**Вопрос 1.7. Какой процент исходных данных (с округлением) мы отфильтровали?**

1. 8
2. 9
3. 10
4. 11

**Ответ:** 3.

### Решение:

In [ ]:
filtered_df = df[(df['ap_lo'] <= df['ap_hi']) & 
                 (df['height'] >= df['height'].quantile(0.025)) &
                 (df['height'] <= df['height'].quantile(0.975)) &
                 (df['weight'] >= df['weight'].quantile(0.025)) & 
                 (df['weight'] <= df['weight'].quantile(0.975))]
print(filtered_df.shape[0] / df.shape[0])

Мы отбросили примерно 10% исходных данных.

# Часть 2. Визуальный анализ данных

## 2.1. Визуализация корреляционной матрицы

Чтобы лучше понять признаки, можно построить матрицу коэффициентов корреляции между ними. Используйте исходный (неотфильтрованный) датасет.

### Задача:

Постройте корреляционную матрицу с помощью [`heatmap()`](http://seaborn.pydata.org/generated/seaborn.heatmap.html). Саму матрицу можно получить стандартными средствами `pandas` с параметрами по умолчанию.

### Решение:

In [ ]:
# Calculate the correlation matrix
df = filtered_df.copy()

corr = df.corr(method='pearson')

# Create a mask to hide the upper triangle of the correlation matrix (which is symmetric)
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(12, 9))

sns.heatmap(corr, mask=mask, vmax=1, center=0, annot=True, fmt='.1f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

**Вопрос 2.1.** Какая пара признаков имеет наибольшую корреляцию Пирсона с признаком *gender*?

1. Cardio, Cholesterol
2. Height, Smoke
3. Smoke, Alco
4. Height, Weight

**Ответ:** 2.

## 2.2. Распределение роста мужчин и женщин

Ранее, изучая уникальные значения, мы увидели, что пол кодируется числами *1* и *2*. Хотя заранее неизвестно, какое значение какому полу соответствует, это можно понять графически, посмотрев на средний рост и вес для каждого значения признака *gender*.

### Задача:

Постройте violin‑plot (виолин‑диаграмму) для роста и пола с помощью [`violinplot()`](https://seaborn.pydata.org/generated/seaborn.violinplot.html). Используйте параметры:
- `hue` — чтобы разделить по полу;
- `scale` — чтобы учитывать количество наблюдений для каждого пола.

Чтобы график корректно отрисовался, нужно перевести `DataFrame` в *длинный* формат с помощью функции `melt()` из `pandas`. В качестве примера можно посмотреть [сюда](https://stackoverflow.com/a/41575149/3338479).

### Решение:

In [ ]:
df_melt = pd.melt(frame=df, value_vars=['height'], id_vars=['gender'])

plt.figure(figsize=(12, 10))
ax = sns.violinplot(
    x='variable', 
    y='value', 
    hue='gender', 
    palette="muted", 
    split=True, 
    data=df_melt, 
    scale='count',
    scale_hue=False
)

### Задача:

Постройте два [`kdeplot`](https://seaborn.pydata.org/generated/seaborn.kdeplot.html) для признака *height* (отдельно по полу) на одном графике. Так различие между полами будет видно более явно, но по нему нельзя будет оценить количество наблюдений в каждой группе.

### Решение:

In [ ]:
sns.FacetGrid(df, hue="gender", height=12) \
   .map(sns.kdeplot, "height").add_legend();

## 2.3. Ранговая корреляция

В большинстве случаев *коэффициента линейной корреляции Пирсона* достаточно, чтобы увидеть зависимости в данных.
Но давайте пойдём чуть дальше и посчитаем [ранговую корреляцию](https://en.wikipedia.org/wiki/Rank_correlation). Она позволит выявить пары признаков, у которых меньший ранг в вариационном ряду одного признака всегда предшествует большему рангу другого (и наоборот для отрицательной корреляции).

### Задача:

Посчитайте и изобразите корреляционную матрицу с использованием [рангового коэффициента корреляции Спирмена](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient).

### Решение:

In [ ]:
# Calculate the correlation matrix
corr = df[['id', 'age', 'height', 'weight', 
           'ap_hi', 'ap_lo', 'cholesterol', 
           'gluc']].corr(method='spearman')

# Create a mask to hide the upper triangle of the correlation matrix (which is symmetric)
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(12, 10))

# Plot the heatmap using the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, vmax=1, center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

**Вопрос 2.2.** Какая пара признаков имеет наибольшую ранговую корреляцию (Спирмена)?

1. Height, Weight
2. Age, Weight
3. Cholesterol, Gluc
4. Cardio, Cholesterol
5. Ap_hi, Ap_lo
6. Smoke, Alco

**Ответ:** 5.

**Вопрос 2.3.** Почему эти признаки имеют высокую ранговую корреляцию?

1. Неточности в данных (ошибки сбора).
2. Связь ложная, эти признаки не должны быть связаны.
3. Природа самих данных.

**Ответ:** 3.

## 2.4. Возраст

Ранее мы уже посчитали возраст респондентов в годах на момент обследования.

### Задача:

Постройте *count‑plot* с помощью [`countplot()`](http://seaborn.pydata.org/generated/seaborn.countplot.html), где по оси *X* откладывается возраст, а по оси *Y* — количество людей. Для каждого возраста на графике должно быть по два столбца — по числу людей каждого класса *cardio* для данного возраста.

### Решение:

In [ ]:
sns.countplot(x="age_years", hue='cardio', data=df);

**Вопрос 2.4.** Начиная с какого минимального возраста количество людей с CVD превышает количество людей без CVD?

1. 44
2. 55
3. 64
4. 70

**Ответ:** 2.